In [1]:
!wget https://raw.githubusercontent.com/omniscientoctopus/Physics-Informed-Neural-Networks/7d26e55cc665b79b652b226860177fa4defe9190/PyTorch/Burgers\'%20Equation/Data/burgers_shock_mu_005_pi.mat

--2025-01-20 07:08:42--  https://raw.githubusercontent.com/omniscientoctopus/Physics-Informed-Neural-Networks/7d26e55cc665b79b652b226860177fa4defe9190/PyTorch/Burgers'%20Equation/Data/burgers_shock_mu_005_pi.mat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 157672 (154K) [application/octet-stream]
Saving to: ‘burgers_shock_mu_005_pi.mat’

burgers_shock_mu_00 100%[===================>] 153.98K  --.-KB/s    in 0.009s  

2025-01-20 07:08:43 (16.6 MB/s) - ‘burgers_shock_mu_005_pi.mat’ saved [157672/157672]



In [68]:
# !wget https://raw.githubusercontent.com/broccubali/PINNs/siddhi/pde-gen/burgers/data/burgerClean.npy
# !wget https://raw.githubusercontent.com/broccubali/PINNs/master/pde-gen/burgers/data/burgerExponential.npy
!wget https://raw.githubusercontent.com/broccubali/PINNs/siddhi/pde-gen/burgers/data/burgerSkewedNormal.npy
!wget https://raw.githubusercontent.com/broccubali/PINNs/master/pde-gen/burgers/data/t_coordinate.npy
!wget https://raw.githubusercontent.com/broccubali/PINNs/master/pde-gen/burgers/data/x_coordinate.npy

--2025-01-20 07:44:51--  https://raw.githubusercontent.com/broccubali/PINNs/siddhi/pde-gen/burgers/data/burgerSkewedNormal.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 823424 (804K) [application/octet-stream]
Saving to: ‘burgerSkewedNormal.npy’

burgerSkewedNormal. 100%[===================>] 804.12K  --.-KB/s    in 0.007s  

2025-01-20 07:44:52 (118 MB/s) - ‘burgerSkewedNormal.npy’ saved [823424/823424]

--2025-01-20 07:44:52--  https://raw.githubusercontent.com/broccubali/PINNs/master/pde-gen/burgers/data/t_coordinate.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP re

# pinn class

In [34]:
import torch
import torch.nn as nn
import numpy as np
from scipy.io import loadmat
from tqdm import tqdm


class PINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                (
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                    if i % 2 == 0
                    else nn.Tanh()
                )
                for i in range(20)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.loss = nn.MSELoss()
        self.lambda2 = nn.Parameter(
            torch.tensor([0.01], dtype=torch.float32, device="cuda")
        )
        self.lambda1 = nn.Parameter(
            torch.tensor([1.0], dtype=torch.float32, device="cuda")
        )
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        self.optimizer.param_groups[0]["params"].append(self.lambda1)
        self.optimizer.param_groups[0]["params"].append(self.lambda2)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        x = xtrain[:, 0]
        t = xtrain[:, 1]
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        u_x_t = torch.autograd.grad(
            u_pred,
            g,
            torch.ones([xtrain.shape[0], 1]).to("cuda"),
            retain_graph=True,
            create_graph=True,
        )[0]
        u_xx_tt = torch.autograd.grad(
            u_x_t, g, torch.ones(xtrain.shape).to("cuda"), create_graph=True
        )[0]
        u_x = u_x_t[:, [0]]
        u_t = u_x_t[:, [1]]
        u_xx = u_xx_tt[:, [0]]
        return self.loss(u_t + (self.lambda1 * u_pred * u_x) - (self.lambda2 * u_xx), fhat)

    def total_loss(self, xtrain, utrain, fhat):
        return self.loss_fn(xtrain, utrain) + self.residual_loss(xtrain, fhat)

    def train_model(self, xtrain, utrain, epochs=1000):
        fhat = torch.zeros(xtrain.shape[0], 1, device="cuda")
        for epoch in tqdm(range(epochs)):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain, fhat)
            loss.backward()
            self.optimizer.step()
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Loss {loss.item()}, Nu {self.lambda2.item()}, Lambda {self.lambda1.item()}")



model = PINN(input_size=2, hidden_size=20, output_size=1).to("cuda")
print(model)

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=20, bias=True)
    (5): Tanh()
    (6): Linear(in_features=20, out_features=20, bias=True)
    (7): Tanh()
    (8): Linear(in_features=20, out_features=20, bias=True)
    (9): Tanh()
    (10): Linear(in_features=20, out_features=20, bias=True)
    (11): Tanh()
    (12): Linear(in_features=20, out_features=20, bias=True)
    (13): Tanh()
    (14): Linear(in_features=20, out_features=20, bias=True)
    (15): Tanh()
    (16): Linear(in_features=20, out_features=20, bias=True)
    (17): Tanh()
    (18): Linear(in_features=20, out_features=20, bias=True)
    (19): Tanh()
    (20): Linear(in_features=20, out_features=1, bias=True)
  )
  (loss): MSELoss()
)


# load data

In [3]:
data = loadmat("../data/burgers_shock_mu_01_pi.mat")
u = data["usol"].T
x = data["x"]
t = data["t"]

# u = np.load("burgerSkewedNormal.npy")
# x = np.load("x_coordinate.npy")
# t = np.load("t_coordinate.npy")[:-1]
x = torch.tensor(x, dtype=torch.float32)
t = torch.tensor(t, dtype=torch.float32)
u = torch.tensor(u, dtype=torch.float32)
# form a meshgrid, where X contains all the x points from -1 to 1, 100 times for 100 time steps.
# T contains all 100 time steps from 0 to 1, 256 times for 256 x points
X, T = np.meshgrid(x, t)
X.shape, T.shape, u.shape

((100, 256), (100, 256), torch.Size([100, 256]))

In [27]:
# !pip install pyDOE

# Train data

In [7]:
from pyDOE import lhs

xtrue = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))
lb = xtrue[0]
ub = xtrue[-1]
idx = np.random.choice(25600, 10000, replace=False)
xtrain = xtrue[idx, :]
utrain = u.flatten()[idx][:, None]
xtrain.shape, utrain.shape

((10000, 2), torch.Size([10000, 1]))

In [12]:
device = torch.device("cuda")
Xtrain = torch.tensor(xtrain, dtype=torch.float32, device=device)
Xtrue = torch.tensor(xtrue, dtype=torch.float32, device=device)
utrain = utrain.to(device)
utrue = u.flatten()[:, None]

In [36]:
model.train_model(Xtrain, utrain, epochs=10000)

  0%|          | 34/10000 [00:00<00:57, 173.84it/s]

Epoch 0, Loss 0.3765890896320343, Nu 0.009826750494539738, Lambda 0.9999733567237854
Gradients of lambda1: tensor([5.2545e-10], device='cuda:0')
Gradients of lambda2: tensor([4.2907e-09], device='cuda:0')


 10%|█         | 1038/10000 [00:05<00:47, 189.05it/s]

Epoch 1000, Loss 0.01671781577169895, Nu 0.005971658509224653, Lambda 0.531352162361145
Gradients of lambda1: tensor([-0.0047], device='cuda:0')
Gradients of lambda2: tensor([0.0024], device='cuda:0')


 20%|██        | 2034/10000 [00:11<00:44, 177.68it/s]

...

 80%|████████  | 8024/10000 [00:44<00:11, 178.15it/s]

Epoch 8000, Loss 0.0011533654760569334, Nu 0.005356267094612122, Lambda 0.9059507250785828
Gradients of lambda1: tensor([-0.0095], device='cuda:0')
Gradients of lambda2: tensor([1.2686], device='cuda:0')


 90%|█████████ | 9035/10000 [00:50<00:05, 185.55it/s]

Epoch 9000, Loss 0.0009527383372187614, Nu 0.005258429329842329, Lambda 0.9197857975959778
Gradients of lambda1: tensor([-0.0125], device='cuda:0')
Gradients of lambda2: tensor([1.8685], device='cuda:0')


100%|██████████| 10000/10000 [00:56<00:00, 177.70it/s]
